<a href="https://colab.research.google.com/github/Arziva/data-pipeline/blob/main/final_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
def read_ids_from_file(file_path):
    with open(file_path, 'r') as file:
        ids = file.read().replace(',', ' ').split()
        return ids

def create_id_string(ids):
    id_string = ','.join(ids[:100])
    remaining_ids = ids[100:]
    return id_string, remaining_ids

def process_ids(file_path):
    all_ids = read_ids_from_file(file_path)
    while all_ids:
        id_string, all_ids = create_id_string(all_ids)
        url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmids="
        url += id_string



        import requests
        r = requests.get(url)

        import json
        s = r.text.strip()

        jsons = []

        start, end = s.find('{'), s.find('}')
        while True:
            try:
                jsons.append(json.loads(s[start:end + 1]))
            except ValueError:
                end = end + 1 + s[end + 1:].find('}')
            else:
                s = s[end + 1:]
                if not s:
                    break
                start, end = s.find('{'), s.find('}')


        data = r.text
        for i in jsons:
            name = i["id"]
            passages = i.get('passages', [])
            entities_list = []

            text = ""

            for passage in passages:
                text += passage.get('text', '') + " "
                annotations = passage.get('annotations', [])

                for annotation in annotations:
                    entity_id = annotation['id']
                    start_offset = annotation['locations'][0]['offset']
                    end_offset = start_offset + annotation['locations'][0]['length']
                    label = annotation['infons'].get('type', '')

                    entity = {
                        'id' : entity_id,
                        'start_offset': start_offset,
                        'end_offset': end_offset,
                        'label': label
                    }

                    entities_list.append(entity)


            output_data = {
                'text': text,
                'entities': entities_list
            }
            json_object = json.dumps(output_data)
            with open("{}_output.jsonl".format(name), "w") as outfile:
                outfile.write(json_object)






# Enter file name
file_path = 'ids.txt'
process_ids(file_path)
